In [ ]:
from __future__ import division
import cPickle as pkl
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
%matplotlib inline

In [2]:
cc = pkl.load(open('../tmp/dump_2009_class_1__raw_data_.pkl','rb'))
rd, ei = cc['raw_data'], cc['extra_info']

In [4]:
lenghts = [len(rd[k]) for k in rd.keys()]

plt.figure(dpi=100)
plt.hist(pos_lenghts, bins=25)
plt.xlabel('sequence length')
plt.ylabel('count');

# Load raw data

In [ ]:
def load_data(dump_files):
    """Extract sequences, extra info and patient id from dump pkl."""
    # Positive raw data load
    raw_data = []
    extra_info = []
    indexes = []
    for f in tqdm(dump_files, desc='Loading dump files'):
        _tmp = pkl.load(open(os.path.join('..', 'tmp', f), 'rb'))
        _raw_data, _extra_info = _tmp['raw_data'], _tmp['extra_info'].set_index('PIN')
        for k in _extra_info.index:
            if len(_raw_data[k])>0: # Skip empty sequences
                raw_data.append(_raw_data[k])
                extra_info.append([_extra_info.loc[k]['SEX'], _extra_info.loc[k]['AGE']])
                indexes.append(k)
            else:
                print('empty {}'.format(k))
    return raw_data, extra_info, indexes

In [ ]:
dump_files = filter(lambda x: 'raw_data' in x, os.listdir('../tmp/'))
positive_dump_files = filter(lambda x: 'class_1' in x, dump_files)
negative_dump_files = filter(lambda x: 'class_0' in x, dump_files)

Sequences should be like `['A', 8, 'M', 0, ...]` where letters are btos and numbers are weeks from one btos to another

In [ ]:
pos_raw_data, pos_extra_info, pos_id = load_data(positive_dump_files)
print(len(pos_id), 'positive samples')

In [ ]:
neg_raw_data, neg_extra_info, neg_id = load_data(negative_dump_files)
print(len(neg_id), 'negative samples')

# EDA

In [ ]:
lenghts = np.array([len(x) for x in pos_BTOS_seq])
pos_idx595 = np.argwhere(np.logical_and(lenghts>=np.percentile(lenghts, 5), lenghts<=np.percentile(lenghts, 95))).ravel()
pos_lenghts = lenghts[pos_idx595]

plt.figure(dpi=100)
plt.hist(pos_lenghts, bins=25)
plt.xlabel('sequence length')
plt.ylabel('count');

In [ ]:
lenghts = np.array([len(x) for x in neg_BTOS_seq])
neg_idx595 = np.argwhere(np.logical_and(lenghts>=np.percentile(lenghts, 5), lenghts<=np.percentile(lenghts, 95))).ravel()
neg_lenghts = lenghts[neg_idx595]

plt.figure(dpi=100)
plt.hist(neg_lenghts, bins=25)
plt.xlabel('sequence length')
plt.ylabel('count');

In [ ]:
import pandas as pd
imap = pd.read_csv('../mbspbs10pc/data/imap.tsv', sep='\t', header=0, usecols=['ITEM', 'BTOS'])
imap.head()

In [ ]:
btos = pd.read_csv('../mbspbs10pc/data/btos_details.csv', header=0)
btos.head()

In [ ]:
tmp = pd.merge(imap, btos, left_on='BTOS', right_on='BTOS (Heirarchy order)', how='left')[['ITEM', 'BTOS_x', 'BTOS_y']]
tmp = tmp.rename({'BTOS_y': 'BTOS', 'BTOS_x': 'BTOS (Heirarchy order)'}, axis=1)
tmp.head()

In [ ]:
tmp.to_csv('../mbspbs10pc/data/imap_derived.csv')

In [ ]:
str('a')

In [ ]:
''.join([])